In [36]:
import pandas as pd
import streamlit as st
import plotly.express as px
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
cores_equatorial = ['#00C0F3','#FABD0A','#2F2E79','#F15B40']
import plotly.graph_objects as go


Instruções: O estudo de caso é uma análise que o candidato deverá realizar seguindo as instruções que estão sendo solicitadas, mas este não deverá ficar restritos somente a elas. Caso o candidato identifique alguma análise complementar que seja importante para fechar a lógica da análise, isso deverá ser feito e demonstrado através de gráficos, ações prazos, etc.                                                                                                                                                             Não existe certo e errado nessa avaliação e o candidato pode ficar a vontade para trabalhar as informações a partir das solicitações abaixo. Boa Sorte!																							
																																								
Desenvolver dashboard que exiba:																														
																														
1) DEC e FEC mensal das Regionais por abrangência																														
2) Pareto de Causas do DEC																														
3) TOP 20 equipamentos mais ofensores do DEC de cada regional e do Piauí como um todo																														
4) Desenvolva uma visão que permita realizar uma busca por equipamento e visualizar: a quantidade de interrupções por mês, o DEC acumulado e o DEC de cada interrupção deste componente ao longo do período.																														
																														
Com base no dashboard desenvolvido responda:

a) Qual a regional com maior contribuição para o indicador DEC?			

b) Qual a principal causa do DEC em cada regional?			

c) Qual a abrangência de maior impacto para o DEC por regional?												

d) Em relação ao FEC, defina um ranking entre as regionais de acordo com a sua performance.		
																												
e) Indique, através da análise do FEC, quais seriam os principais conjuntos de cada regional onde devemos atuar para redução do indicador e qual o ganho esperado após a atuação.																					

In [3]:
df = pd.read_excel('ESTUDO DE CASO DEC - TATICA.xlsx',sheet_name=1)

In [4]:
df.head()

,OCORRENCIA_ID,DATA,DIA,MES,ANO,DATA_INICIO,TEMPO,SE_AL,OCO_INTERRUPCAO,CAUSA,DEFEITO,OCO_NUMERO,SE,AL,EQUIPAMENTO,TIPO,ABRANGENCIA,NR,CLI_INT,CHI,FEC,DEC,REGIONAL,CONJUNTO,MÊS01
0,52921651.0,13/07/2024,13.0,7.0,2024.0,2024-07-13 21:23:42,0.544722,CPM-01C1,S,FALHA DE CONEXÃO,JUMPER BT,127109.0,CPM,01C1,NaN,CR,CR,1.0,1.0,1.0,6.791508e-07,3.699485e-07,METROPOLITANA,CAMPO MAIOR,7.0
1,52921650.0,13/07/2024,13.0,7.0,2024.0,2024-07-13 19:26:33,0.519722,CPM-01C1,S,FALHA DE CONEXÃO,JUMPER BT,127108.0,CPM,01C1,NaN,CR,CR,1.0,1.0,1.0,6.791508e-07,3.529698e-07,METROPOLITANA,CAMPO MAIOR,7.0
2,52921649.0,13/07/2024,13.0,7.0,2024.0,2024-07-13 20:45:21,0.575833,CPM-01C1,S,FALHA DE CONEXÃO,JUMPER BT,127107.0,CPM,01C1,NaN,CR,CR,1.0,1.0,1.0,6.791508e-07,3.910777e-07,METROPOLITANA,CAMPO MAIOR,7.0
3,52921648.0,13/07/2024,13.0,7.0,2024.0,2024-07-13 19:25:32,0.572222,CPM-01C1,S,FALHA DE CONEXÃO,JUMPER BT,127106.0,CPM,01C1,NaN,CR,CR,1.0,1.0,1.0,6.791508e-07,3.886252e-07,METROPOLITANA,CAMPO MAIOR,7.0
4,52921424.0,06/07/2024,6.0,7.0,2024.0,2024-07-06 11:06:53,0.513333,CMP-01C2,S,FALHA DE CONEXÃO,CONECTOR,126882.0,CMP,01C2,NaN,CR,CR,1.0,1.0,1.0,6.791508e-07,3.486308e-07,NORTE,CAMURUPIM,7.0


In [5]:
df.shape

(155856, 25)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155856 entries, 0 to 155855
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   OCORRENCIA_ID    155853 non-null  float64       
 1   DATA             155853 non-null  object        
 2   DIA              155853 non-null  float64       
 3   MES              155853 non-null  float64       
 4   ANO              155853 non-null  float64       
 5   DATA_INICIO      155853 non-null  datetime64[ns]
 6   TEMPO            155853 non-null  float64       
 7   SE_AL            155853 non-null  object        
 8   OCO_INTERRUPCAO  151647 non-null  object        
 9   CAUSA            155853 non-null  object        
 10  DEFEITO          155853 non-null  object        
 11  OCO_NUMERO       155853 non-null  float64       
 12  SE               155276 non-null  object        
 13  AL               154957 non-null  object        
 14  EQUIPAMENTO      574

In [7]:
df.columns

Index(['OCORRENCIA_ID', 'DATA', 'DIA', 'MES', 'ANO', 'DATA_INICIO', 'TEMPO',
       'SE_AL', 'OCO_INTERRUPCAO', 'CAUSA', 'DEFEITO', 'OCO_NUMERO', 'SE',
       'AL', 'EQUIPAMENTO', 'TIPO', 'ABRANGENCIA', 'NR', 'CLI_INT', 'CHI',
       'FEC', 'DEC', 'REGIONAL', 'CONJUNTO', 'MÊS01'],
      dtype='object')

In [8]:
df['ABRANGENCIA'].value_counts()

ABRANGENCIA
CR    98184
TF    32699
CH    24541
AL      411
SE       18
Name: count, dtype: int64

In [9]:
# Substituindo o sigla da abrangencia pelo nome
df['ABRANGENCIA'] = df['ABRANGENCIA'].replace(['CR'],'Cliente')
df['ABRANGENCIA'] = df['ABRANGENCIA'].replace(['TF'],'Transformador')
df['ABRANGENCIA'] = df['ABRANGENCIA'].replace(['CH'],'Chave')
df['ABRANGENCIA'] = df['ABRANGENCIA'].replace(['AL'],'Alimentador')
df['ABRANGENCIA'] = df['ABRANGENCIA'].replace(['SE'],'Subestação')

In [10]:
df['DATA']

0         13/07/2024
1         13/07/2024
2         13/07/2024
3         13/07/2024
4         06/07/2024
             ...    
155851    02/01/2024
155852    02/01/2024
155853           NaN
155854           NaN
155855           NaN
Name: DATA, Length: 155856, dtype: object

In [11]:
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y')

In [12]:
# Agrupar por REGIONAL e DATA (mensal)
df_def_fec = df.groupby(['REGIONAL', pd.Grouper(key='DATA', freq='M')])[['DEC','FEC']].sum()

# Resetar o índice para transformar em DataFrame
df_def_fec = df_def_fec.reset_index()

# Formatar a coluna DATA para mostrar apenas o mês e o ano
df_def_fec['DATA'] = df_def_fec['DATA'].dt.to_period('M').astype(str)

# Renomear colunas conforme necessário
df_def_fec.columns = ['REGIONAL', 'MES', 'DEC','FEC']

# Dicionário de meses em português
meses_pt = {
    '2024-01': 'Janeiro',
    '2024-02': 'Fevereiro',
    '2024-03': 'Março',
    '2024-04': 'Abril',
    '2024-05': 'Maio',
    '2024-06': 'Junho',
    '2024-07': 'Julho',
    '2024-08': 'Agosto',
    '2024-09': 'Setembro',
    '2024-10': 'Outubro',
    '2024-11': 'Novembro',
    '2024-12': 'Dezembro'
}

# Substituir os valores na coluna 'MES' com os nomes dos meses em português
df_def_fec['MES'] = df_def_fec['MES'].map(meses_pt)

C:\Users\Victor\AppData\Local\Temp\ipykernel_9000\1015987261.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_def_fec = df.groupby(['REGIONAL', pd.Grouper(key='DATA', freq='M')])[['DEC','FEC']].sum()


In [13]:
# Cores personalizadas
cores_personalizadas = ['#00C0F3', '#FABD0A', '#2F2E79', '#F15B40']

# Plotar o gráfico de linhas usando Plotly Express
fig = px.line(df_def_fec, x='MES', y='DEC', color='REGIONAL',
              color_discrete_sequence=cores_personalizadas,
              title='DEC por Regional ao Longo do Tempo',
              labels={'MES': 'Mês', 'DEC': 'DEC'},
              height=400)

# Adicionar marcadores e configurar grids
fig.update_traces(mode='lines+markers')  # Adicionar marcadores aos pontos
fig.update_layout(
    xaxis=dict(
        title='Mês',
        showgrid=False,  # Remover grid do eixo x
        tickmode='array',
        tickvals=list(meses_pt.values()),  # Definir valores dos ticks
        ticktext=list(meses_pt.values())  # Definir textos dos ticks
    ),
    yaxis=dict(
        title='DEC',
        showgrid=True,  # Mostrar grid do eixo y
        gridcolor='LightGray',  # Cor do grid
        gridwidth=0.5  # Largura do grid
    )
)

# Exibir o gráfico
fig.show()

In [14]:
# Cores personalizadas
cores_personalizadas = ['#00C0F3', '#FABD0A', '#2F2E79', '#F15B40']

# Plotar o gráfico de linhas usando Plotly Express
fig = px.line(df_def_fec, x='MES', y='FEC', color='REGIONAL',
              color_discrete_sequence=cores_personalizadas,
              title='FEC Mensal por Regional',
              labels={'MES': 'Mês', 'FEC': 'FEC'},
              height=400)

# Adicionar marcadores e configurar grids
fig.update_traces(mode='lines+markers')  # Adicionar marcadores aos pontos
fig.update_layout(
    xaxis=dict(
        title='Mês',
        showgrid=False,  # Remover grid do eixo x
        tickmode='array',
        tickvals=list(meses_pt.values()),  # Definir valores dos ticks
        ticktext=list(meses_pt.values())  # Definir textos dos ticks
    ),
    yaxis=dict(
        title='FEC',
        showgrid=True,  # Mostrar grid do eixo y
        gridcolor='LightGray',  # Cor do grid
        gridwidth=0.5  # Largura do grid
    )
)

# Exibir o gráfico
fig.show()

In [15]:
df.CAUSA.value_counts()

CAUSA
FALHA DE CONEXÃO                            67449
ARVORE NA REDE                              22317
CAUSA NÃO IDENTIFICADA                       9911
ANIMAL NO CIRCUITO                           8914
SOBRECARGA                                   8362
CONDUTOR PARTIDO                             8350
OBJETO NA REDE                               8305
RAMAL PARTIDO                                7101
REDE TRANÇADA                                2266
TRANSFORMADOR AVARIADO                       2234
INCENDIO                                     1774
DESLIGAMENTO PROGRAMADO COLETIVO             1632
DEFEITO EM MEDIDOR                           1598
POSTE AVARIADO / ADERNADO                    1175
DEFEITO EM ISOLADOR                          1029
RAMAL TRANÇADO                                770
FALHA EM EQUIPAMENTO                          717
ELO FUSIVEL INADEQUADO                        592
FURTO                                         344
FALHA OPERACIONAL                           

In [22]:
# Agrupar por REGIONAL e DATA (mensal)
df_causa = df.groupby('CAUSA' )[['DEC']].sum().sort_values(by='DEC',ascending=False)
df_causa.reset_index(inplace=True)

In [26]:
df_causa['DEC'].sum()

np.float64(21.88381528610775)

In [34]:
df_causa['PERCENTUAL'] = (df_causa['DEC'] / df_causa['DEC'].sum()) * 100
df_causa['PERCENTUAL ACUMULADO'] = df_causa['PERCENTUAL'].cumsum() / df_causa['PERCENTUAL'].sum() * 100

In [33]:
df_causa['PERCENTUAL'].head(7).sum()

np.float64(80.91765837986709)

In [73]:
import plotly.graph_objects as go

# Criar o gráfico de barras para o DEC por causa
fig = go.Figure()

# Adicionar barras sem rótulos de dados
fig.add_trace(go.Bar(
    x=df_causa['CAUSA'],
    y=df_causa['DEC'],
    name='DEC',
    marker=dict(color='#00C0F3'),
    textposition='none'  # Remove os rótulos de dados das barras
))

# Adicionar linha de Percentual Acumulado com rótulos condicionais
fig.add_trace(go.Scatter(
    x=df_causa['CAUSA'],
    y=df_causa['PERCENTUAL ACUMULADO'],
    name='Percentual Acumulado',
    yaxis='y2',
    mode='lines+markers+text',
    line=dict(color='#FABD0A', width=2),
    marker=dict(size=8),  # Tamanho dos marcadores
    text=[f'{val:.2f}%' if val <= 81 else '' for val in df_causa['PERCENTUAL ACUMULADO']],  # Adiciona rótulos com porcentagem até 81%
    textposition='top center',  # Posiciona os rótulos acima dos pontos
    textfont=dict(size=12),  # Ajusta o tamanho da fonte dos rótulos
))

# Atualizar layout para ajustar espaçamento, tamanho de fonte e largura do gráfico
fig.update_layout(
    title='Gráfico de Pareto - DEC por Causa',
    title_font=dict(size=18),
    xaxis=dict(
        
        title_font=dict(size=14),
        tickangle=-70,  # Gira os rótulos do eixo X em 90 graus
        tickfont=dict(size=12),  # Ajusta o tamanho da fonte dos rótulos do eixo X
    ),
    yaxis=dict(
        title='DEC',
        title_font=dict(size=14),
        tickfont=dict(size=12),  # Ajusta o tamanho da fonte dos rótulos do eixo Y
        gridcolor='rgba(0,0,0,0)'  # Remove as grades verticais
    ),
    yaxis2=dict(
        title='Percentual Acumulado',
        overlaying='y',
        side='right',
        range=[0, 110],  # Escala da porcentagem ajustada de 0 a 100%
        title_font=dict(size=14),
        tickfont=dict(size=12),  # Ajusta o tamanho da fonte dos rótulos do eixo Y2
        showgrid=True,  # Exibe a grade horizontal para o eixo Y2
        gridcolor='rgba(211, 211, 211, 0.5)'  # Define a cor e a opacidade da grade horizontal
    ),
    width=1090,  # Aumenta a largura do gráfico
    margin=dict(l=60, r=60, t=80, b=120),  # Ajusta as margens do gráfico
    showlegend=False
)

# Exibir o gráfico
fig.show()


In [83]:
import plotly.graph_objects as go

# Criar o gráfico de barras para o DEC por causa
fig = go.Figure()

# Adicionar barras sem rótulos de dados
fig.add_trace(go.Bar(
    x=df_causa['CAUSA'],
    y=df_causa['DEC'],
    name='DEC',
    marker=dict(color='#00C0F3'),
    textposition='none'  # Remove os rótulos de dados das barras
))

# Adicionar linha de Percentual Acumulado com rótulos condicionais
fig.add_trace(go.Scatter(
    x=df_causa['CAUSA'],
    y=df_causa['PERCENTUAL ACUMULADO'],
    name='Percentual Acumulado',
    yaxis='y2',
    mode='lines+markers+text',
    line=dict(color='#FABD0A', width=2),
    marker=dict(size=8),  # Tamanho dos marcadores
    text=[f'{val:.2f}%' if val <= 81 else '' for val in df_causa['PERCENTUAL ACUMULADO']],  # Adiciona rótulos com porcentagem até 81%
    textposition='top center',  # Posiciona os rótulos acima dos pontos
    textfont=dict(size=12),  # Ajusta o tamanho da fonte dos rótulos
))

# Atualizar layout para ajustar espaçamento, tamanho de fonte e largura do gráfico
fig.update_layout(
    title='Gráfico de Pareto - DEC por Causa',
    title_font=dict(size=18),
    xaxis=dict(
        title='Causa',
        title_font=dict(size=14),
        tickangle=-40,  # Ajusta o ângulo dos rótulos do eixo X para melhor legibilidade
        tickfont=dict(size=12),  # Ajusta o tamanho da fonte dos rótulos do eixo X
        tickmode='array',
        tickvals=df_causa['CAUSA'],
        ticktext=df_causa['CAUSA'],  # Exibe o texto completo dos rótulos
    ),
    yaxis=dict(
        title='DEC',
        title_font=dict(size=14),
        tickfont=dict(size=12),  # Ajusta o tamanho da fonte dos rótulos do eixo Y
        gridcolor='rgba(0,0,0,0)'  # Remove as grades verticais
    ),
    yaxis2=dict(
        title='Percentual Acumulado',
        overlaying='y',
        side='right',
        range=[0, 110],  # Aumenta a faixa do eixo Y2 para até 110%
        title_font=dict(size=14),
        tickfont=dict(size=12),  # Ajusta o tamanho da fonte dos rótulos do eixo Y2
        showgrid=True,  # Exibe a grade horizontal para o eixo Y2
        gridcolor='rgba(211, 211, 211, 0.5)'  # Define a cor e a opacidade da grade horizontal
    ),
    width=1100,  # Aumenta a largura do gráfico para melhor visualização
    height=700,  # Ajusta a altura do gráfico para permitir mais espaço na parte inferior
    margin=dict(l=80, r=80, t=80, b=180),  # Ajusta as margens do gráfico para dar mais espaço na parte inferior
    showlegend=False
)

# Exibir o gráfico
fig.show()


In [77]:
df.head()

,OCORRENCIA_ID,DATA,DIA,MES,ANO,DATA_INICIO,TEMPO,SE_AL,OCO_INTERRUPCAO,CAUSA,DEFEITO,OCO_NUMERO,SE,AL,EQUIPAMENTO,TIPO,ABRANGENCIA,NR,CLI_INT,CHI,FEC,DEC,REGIONAL,CONJUNTO,MÊS01
0,52921651.0,2024-07-13,13.0,7.0,2024.0,2024-07-13 21:23:42,0.544722,CPM-01C1,S,FALHA DE CONEXÃO,JUMPER BT,127109.0,CPM,01C1,NaN,CR,Cliente,1.0,1.0,1.0,6.791508e-07,3.699485e-07,METROPOLITANA,CAMPO MAIOR,7.0
1,52921650.0,2024-07-13,13.0,7.0,2024.0,2024-07-13 19:26:33,0.519722,CPM-01C1,S,FALHA DE CONEXÃO,JUMPER BT,127108.0,CPM,01C1,NaN,CR,Cliente,1.0,1.0,1.0,6.791508e-07,3.529698e-07,METROPOLITANA,CAMPO MAIOR,7.0
2,52921649.0,2024-07-13,13.0,7.0,2024.0,2024-07-13 20:45:21,0.575833,CPM-01C1,S,FALHA DE CONEXÃO,JUMPER BT,127107.0,CPM,01C1,NaN,CR,Cliente,1.0,1.0,1.0,6.791508e-07,3.910777e-07,METROPOLITANA,CAMPO MAIOR,7.0
3,52921648.0,2024-07-13,13.0,7.0,2024.0,2024-07-13 19:25:32,0.572222,CPM-01C1,S,FALHA DE CONEXÃO,JUMPER BT,127106.0,CPM,01C1,NaN,CR,Cliente,1.0,1.0,1.0,6.791508e-07,3.886252e-07,METROPOLITANA,CAMPO MAIOR,7.0
4,52921424.0,2024-07-06,6.0,7.0,2024.0,2024-07-06 11:06:53,0.513333,CMP-01C2,S,FALHA DE CONEXÃO,CONECTOR,126882.0,CMP,01C2,NaN,CR,Cliente,1.0,1.0,1.0,6.791508e-07,3.486308e-07,NORTE,CAMURUPIM,7.0


In [98]:
df.EQUIPAMENTO = df.EQUIPAMENTO.astype(str)

In [143]:
# Agrupar por EQUIPAMENTO e somar os valores de DEC
df_equipamentos = df.groupby('EQUIPAMENTO')[['DEC']].sum().sort_values(by='DEC', ascending=False)
df_equipamentos.reset_index(inplace=True)

# Substituir a string 'nan' por NaN
df_equipamentos['EQUIPAMENTO'].replace('nan', pd.NA, inplace=True)

# Remover linhas com NaN na coluna 'EQUIPAMENTO'
df_equipamentos = df_equipamentos.dropna(subset=['EQUIPAMENTO'])
df_equipamentos['DEC'] = df_equipamentos['DEC'].round(4)
# Garantir que a coluna 'EQUIPAMENTO' seja tratada como uma categoria
df_equipamentos['EQUIPAMENTO'] = df_equipamentos['EQUIPAMENTO'].astype('category')
df_equipamentos.head(20)


C:\Users\Victor\AppData\Local\Temp\ipykernel_9000\3262054663.py:6: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,EQUIPAMENTO,DEC
1,PRI01Y1,0.1217
2,1555235,0.0997
3,VAL01C1,0.0785
4,MTO01C3,0.0655
5,SPP09F5,0.0583
6,1751964,0.0534
7,1602012,0.0494
8,1631942,0.0490
9,1567160,0.0485
10,1736728,0.0485


In [144]:
# Garanta que a coluna 'EQUIPAMENTO' seja tratada como string
df_equipamentos['EQUIPAMENTO'] = df_equipamentos['EQUIPAMENTO'].astype(str)

# Garanta que a coluna 'EQUIPAMENTO' seja tratada como uma categoria
df_equipamentos['EQUIPAMENTO'] = pd.Categorical(df_equipamentos['EQUIPAMENTO'], categories=df_equipamentos['EQUIPAMENTO'].unique())

# Encontre o valor máximo de DEC
max_dec = df_equipamentos['DEC'].max()

# Crie o gráfico de barras com rótulos de dados
fig = px.bar(df_equipamentos.head(20), x='EQUIPAMENTO', y='DEC', 
             title='Valores de DEC por Equipamento',
             labels={'EQUIPAMENTO': 'Nome do Equipamento', 'DEC': 'Valor do DEC'},
             text='DEC',  # Adiciona os rótulos de dados
             color_discrete_sequence=['#2F2E79'])  # Define uma cor uniforme para todas as barras

# Ajuste o layout do gráfico
fig.update_layout(
    xaxis_title='Nome do Equipamento',
    yaxis_title='Valor do DEC',
    xaxis_tickangle=-45,  # Inclina os rótulos do eixo x para melhor leitura
    yaxis=dict(range=[0, max_dec * 1.3]),  # Define o intervalo do eixo y
    xaxis_type='category',  # Força o eixo x a ser categórico
    bargap=0.2  # Ajusta a largura das barras (reduza o valor para barras mais largas)
)

# Ajuste a aparência dos rótulos de dados
fig.update_traces(
    texttemplate='%{text:.3f}',  # Ajusta a precisão dos rótulos
    textposition='inside',  # Coloca os rótulos dentro das barras
    textfont=dict(color='white', size=12),  # Define a cor dos rótulos como branca e aumenta o tamanho da fonte
    textangle=0  # Define a rotação dos rótulos para horizontal
)

fig.show()

In [145]:
df_equipamentos = df_equipamentos.head(20).sort_values(by='DEC',ascending=True)
df_equipamentos['EQUIPAMENTO'] = df_equipamentos['EQUIPAMENTO'].astype(str)

# Garanta que a coluna 'EQUIPAMENTO' seja tratada como uma categoria
df_equipamentos['EQUIPAMENTO'] = pd.Categorical(df_equipamentos['EQUIPAMENTO'], categories=df_equipamentos['EQUIPAMENTO'].unique())

# Encontre o valor máximo de DEC
max_dec = df_equipamentos['DEC'].max()

# Crie o gráfico de barras horizontais com rótulos de dados
fig = px.bar(df_equipamentos.tail(20), x='DEC', y='EQUIPAMENTO', 
             title='Valores de DEC por Equipamento',
             labels={'EQUIPAMENTO': 'Nome do Equipamento', 'DEC': 'Valor do DEC'},
             text='DEC',  # Adiciona os rótulos de dados
             color_discrete_sequence=['#2F2E79'],  # Define uma cor uniforme para todas as barras
             orientation='h')  # Define a orientação horizontal das barras

# Ajuste o layout do gráfico
fig.update_layout(
    xaxis_title='Valor do DEC',
    yaxis_title='Nome do Equipamento',
    xaxis=dict(range=[0, max_dec * 1.1]),  # Define o intervalo do eixo x
    yaxis_type='category',  # Força o eixo y a ser categórico
    xaxis_tickformat='.3f',  # Formato dos rótulos do eixo x
    bargap=0.1,  # Diminui a largura das barras (ajuste conforme necessário)
    height=800,  # Aumenta a altura do gráfico para melhor visualização
    width=600  # Aumenta a largura do gráfico para melhor visualização
)

# Ajuste a aparência dos rótulos de dados
fig.update_traces(
    texttemplate='%{text:.3f}',  # Ajusta a precisão dos rótulos
    textposition='inside',  # Coloca os rótulos dentro das barras
    textfont=dict(color='white', size=12),  # Define a cor dos rótulos como branca e aumenta o tamanho da fonte
    textangle=0  # Define a rotação dos rótulos para horizontal
)

fig.show()


 Desenvolva uma visão que permita realizar uma busca por equipamento e visualizar: a quantidade de interrupções por mês, o DEC acumulado e o DEC de cada interrupção deste componente ao longo do período.

In [263]:
df_pergunta4 = df[['OCORRENCIA_ID','DATA','EQUIPAMENTO','DEC']]
teste = df_pergunta4[df_pergunta4['EQUIPAMENTO'] == 'VAL01C1']

In [251]:
# Agrupar por REGIONAL e DATA (mensal)
df_num_interrup_equipamentos = teste.groupby( pd.Grouper(key='DATA', freq='M'))['OCORRENCIA_ID'].count().reset_index()
df_num_interrup_equipamentos.columns = ['DATA','CONTAGEM']

# Formatar a coluna DATA para mostrar apenas o mês e o ano
df_num_interrup_equipamentos['MES'] = df_num_interrup_equipamentos['DATA'].dt.to_period('M').astype(str)

# Dicionário de meses em português
meses_pt = {
    '2024-01': 'Janeiro',
    '2024-02': 'Fevereiro',
    '2024-03': 'Março',
    '2024-04': 'Abril',
    '2024-05': 'Maio',
    '2024-06': 'Junho',
    '2024-07': 'Julho',
    '2024-08': 'Agosto',
    '2024-09': 'Setembro',
    '2024-10': 'Outubro',
    '2024-11': 'Novembro',
    '2024-12': 'Dezembro'
}


# Substituir os valores na coluna 'MES' com os nomes dos meses em português
df_num_interrup_equipamentos['MES'] = df_num_interrup_equipamentos['MES'].map(meses_pt)

C:\Users\Victor\AppData\Local\Temp\ipykernel_9000\646531557.py:2: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [252]:
# Encontre o valor máximo de CONTAGEM
max_contagem = df_num_interrup_equipamentos['CONTAGEM'].max()

# Crie o gráfico de linha por mês com marcadores e rótulos de dados
fig = px.line(df_num_interrup_equipamentos, x='MES', y='CONTAGEM', 
             title='Quantidade de Interrupções por Mês',
             #labels={'MES': 'Mês', 'CONTAGEM': 'Número de Interrupções'},
             color_discrete_sequence=['#2F2E79'],  # Define uma cor uniforme para a linha
             text='CONTAGEM')  # Adiciona os rótulos de dados

# Ajuste o layout do gráfico
fig.update_layout(
    xaxis_title='Mês',
    yaxis_title='Número de Interrupções',
    xaxis_tickangle=0,  # Inclina os rótulos do eixo x para melhor leitura
    yaxis=dict(range=[0, max_contagem * 1.3]),  # Define o intervalo do eixo y
    height=400,  # Ajusta a altura do gráfico
    width=800  # Ajusta a largura do gráfico
)

# Adicione marcadores e rótulos de dados à linha
fig.update_traces(
    mode='lines+markers+text',  # Adiciona marcadores e rótulos de dados
    marker=dict(size=8, color='#2F2E79', line=dict(width=2, color='#2F2E79')),  # Estilo dos marcadores
    line=dict(width=3),  # Define a largura da linha
    textposition='top center',  # Posição dos rótulos de dados
    texttemplate='%{text:.0f}'  # Formato dos rótulos de dados (número inteiro)
)

fig.show()

In [264]:
df_pergunta4 = df[['OCORRENCIA_ID','DATA','EQUIPAMENTO','DEC']]
df_dec_acumulado = df_pergunta4[['DATA','DEC']]
df_dec_acumulado.set_index('DATA',inplace=True)

In [265]:
# Preparar dados para o DEC acumulado
df_dec_acumulado = teste[['DATA', 'DEC']].copy()
df_dec_acumulado.set_index('DATA', inplace=True)
df_dec_acumulado = df_dec_acumulado.sort_index()
df_dec_acumulado['DEC_ACUMULADO'] = df_dec_acumulado['DEC'].cumsum()  # Calcula o DEC acumulado

# Preparar dados para DEC de cada interrupção
df_dec_interrupcao = teste.copy()
df_dec_interrupcao['DATA'] = pd.to_datetime(df_dec_interrupcao['DATA'])
df_dec_interrupcao.set_index('DATA', inplace=True)
df_dec_interrupcao = df_dec_interrupcao.sort_index()

# Combine os dados
df_combined = df_dec_acumulado[['DEC_ACUMULADO']].copy()
df_combined['DEC_INTERUPCAO'] = df_dec_interrupcao['DEC']

# Crie o gráfico de cascata (waterfall)
fig = go.Figure(go.Waterfall(
    name="DEC Acumulado",
    orientation="v",
    measure=["relative"] * len(df_combined),  # "relative" indica que os valores são incrementais
    x=df_combined.index.strftime('%d/%m/%Y'),  # Datas como rótulos
    textposition="outside",
    text=df_combined['DEC_INTERUPCAO'].round(2).astype(str),  # Texto das barras
    y=df_combined['DEC_INTERUPCAO'],  # Valores das interrupções
    connector={"line": {"color": "rgba(63, 63, 63, 0.7)"}},  # Linha de conexão entre barras
    increasing={"marker": {"color": "#FABD0A"}},  # Cor das barras para DEC
    decreasing={"marker": {"color": "#F15B40"}},  # Cor das barras negativas (não aplicável aqui)
    totals={"marker": {"color": "#2F2E79"}}  # Cor do total (não aplicável aqui)
))

# Ajuste o layout do gráfico
fig.update_layout(
    title="DEC Acumulado e DEC de Cada Interrupção",
    xaxis_title="Data",
    yaxis_title="Valor do DEC",
    height=400,
    width=800,
    showlegend=False
)

fig.show()

In [266]:
# Preparar dados para o DEC acumulado
df_dec_acumulado = teste[['DATA', 'DEC']].copy()
df_dec_acumulado.set_index('DATA', inplace=True)
df_dec_acumulado = df_dec_acumulado.sort_index()
df_dec_acumulado['DEC_ACUMULADO'] = df_dec_acumulado['DEC'].cumsum()  # Calcula o DEC acumulado

# Preparar dados para DEC de cada interrupção
df_dec_interrupcao = teste.copy()
df_dec_interrupcao['DATA'] = pd.to_datetime(df_dec_interrupcao['DATA'])
df_dec_interrupcao.set_index('DATA', inplace=True)
df_dec_interrupcao = df_dec_interrupcao.sort_index()

# Combine os dados
df_combined = df_dec_acumulado[['DEC_ACUMULADO']].copy()
df_combined['DEC_INTERUPCAO'] = df_dec_interrupcao['DEC']

# Crie o gráfico de cascata (waterfall)
fig = go.Figure()

# Adiciona o gráfico de cascata
fig.add_trace(go.Waterfall(
    name="DEC por Interrupção",
    orientation="v",
    measure=["relative"] * len(df_combined),  # "relative" indica que os valores são incrementais
    x=df_combined.index.strftime('%d/%m/%Y'),  # Datas como rótulos
    textposition="outside",
    text=df_combined['DEC_INTERUPCAO'].round(2).astype(str),  # Texto das barras
    y=df_combined['DEC_INTERUPCAO'],  # Valores das interrupções
    connector={"line": {"color": "rgba(63, 63, 63, 0.7)"}},  # Linha de conexão entre barras
    increasing={"marker": {"color": "#FABD0A"}},  # Cor das barras para DEC
    decreasing={"marker": {"color": "#F15B40"}},  # Cor das barras negativas (não aplicável aqui)
    totals={"marker": {"color": "#2F2E79"}}  # Cor do total (não aplicável aqui)
))

# Adiciona a linha de DEC acumulado
fig.add_trace(go.Scatter(
    x=df_combined.index.strftime('%d/%m/%Y'),
    y=df_combined['DEC_ACUMULADO'],
    mode='lines+markers+text',
    name='DEC Acumulado',
    line=dict(color='#2F2E79', width=4),  # Aumenta a espessura da linha e define a cor
    text=df_combined['DEC_ACUMULADO'].round(2).astype(str),  # Rótulo de dados
    textposition='top center'  # Posição do rótulo de dados
))

# Ajuste o layout do gráfico
fig.update_layout(
    title="DEC Acumulado e DEC de Cada Interrupção",
    xaxis_title="Data",
    yaxis_title="Valor do DEC",
    height=600,
    width=1000,
    showlegend=True
)

fig.show()